# <div style=" text-align: center; font-size: 50px; font-weight: bold">Phase 01: Collecting data </div>

## **Import thư viện**

In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
# !pip install selenium
from selenium import webdriver
import re

## **Thu thập dữ liệu bất động sản:**
Trong phần này, chúng ta sẽ thu thập dữ liệu về các bất động sản  từ trang web: https://batdongsan.com.vn/. Trang web này chứa phần lớn các quảng cáo về bất động sản (cho thuê hoặc bán) tại Việt Nam. Trong phạm vi của dự án này, chúng ta sẽ tập trung vào dữ liệu các bất động sản đang bán, nằm tại Thành phố Hồ Chí Minh.


### **1.**

Tạo một DataFrame rỗng để lưu trữ dữ liệu bất động sản. DataFrame sẽ chứa một số đặc điểm như: Address (Địa chỉ), Type (Loại), Area (Diện tích), Price (Giá), Bedroom (Phòng ngủ), Toilet (Phòng vệ sinh), Floor (Tầng), Furniture (Nội thất), Direction (Hướng), Legal (Pháp lý), Posting date (Ngày đăng), Expiry date (Ngày hết hạn), Ad type (Loại quảng cáo), Ad code (Mã quảng cáo).

In [2]:
fields = [ 'Address', 'Type', 'Area', 'Price', 'Bedroom', 'Toilet', 'Floor', 'Furniture', 'Direction','Legal', 'Posting date', 'Expiry date', 'Ad type', 'Ad code']

# Create a empty data frame with these fields
df = pd.DataFrame(columns = fields)

### **2. Thu thập các liên kết đến từng trang web bất động sản:**

Các liên kết được thu thập từ nhiều trang, với mỗi trang chứa nhiều liên kết đến các căn hộ. Những liên kết này sẽ được lưu trong một tệp có tên real_estate_for_sale_links.txt.

Với phần này, chúng ta có thể sử dụng Selenium để truy cập trang web và thu thập dữ liệu từ đó. Liên kết đến trang chủ và tên tệp được cung cấp dưới đây:  

In [3]:
file_name = "../Data/real_estate_for_sale_links.txt"
url = 'https://batdongsan.com.vn/nha-dat-ban-tp-hcm'

In [4]:
def GetTotalPage(url):

    # Create an instance of the driver
    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # The parsing data process
    unprocessed_num_page = soup.find(class_ = 're__pagination-group')
    driver.quit()
    # Regex to find all the current numpages in the screen.
    regex = r'[^\s]+'
    matchs = re.findall(regex, unprocessed_num_page.text)
    new_array = []
    
    for match in matchs:
        new_array.append(int(match.replace('.', '')))

    total_page = max(new_array)

    return total_page

In [ ]:
# Get num of total pages in the website.
total_page = GetTotalPage(url)

Thu thập các đường dẫn đến trang web bất động sản

In [ ]:
def CollectRealEstateForSaleLink (url, page):
    with open (file_name, 'w') as file:
        # Acess to all page
        for i in range(1, page + 1):
            driver = webdriver.Chrome()
            driver.get(url+'/p'+ str(i))
            soup = soup = BeautifulSoup(driver.page_source, 'html.parser')
            apartment_list = soup.find_all("a", class_= "js__product-link-for-product-id")

            # Add all links to the file
            for apartment in apartment_list:
                link = 'https://batdongsan.com.vn/'+ apartment['href']
                file.write(link)
                file.write('\n')

            driver.quit()

In [ ]:
CollectRealEstateForSaleLink (url, total_page)

Đọc file `real_estate_for_sale_links.txt` và lưu các đường links vào mảng


In [5]:
urls = []
with open(file_name, 'r') as file:
    for line in file.readlines():
            urls.append(line)

In [6]:
len(urls)

65358

### **3. Thu thập dữ liệu bất động sản:**

In [7]:
key_map = {
    'Địa chỉ:' : 'Address',
    'Diện tích': 'Area',
    'Mức giá': 'Price',
    'Hướng nhà': 'Direction',
    'Số tầng': 'Floor',
    'Số phòng ngủ': 'Bedroom',
    'Số toilet': 'Toilet',
    'Pháp lý': 'Legal',
    'Nội thất': 'Furniture',
    'Ngày đăng': 'Posting date',
    'Ngày hết hạn': 'Expiry date',
    'Loại tin': 'Ad type',
    'Mã tin': 'Ad code'
}

In [8]:
sale_type = ['Căn hộ chung cư', 'Nhà riêng', 'Nhà biệt thự, liền kề', 'Nhà mặt phố', 'Shophouse nhà phố thương mại', 'Đất nền dự án', 'Đất bán', 'Trang trại, khu nghỉ dưỡng', 'Kho, nhà xưởng','Bất động sản khác','Condotel','kháckhác']
sale_type_dict = {
    '324': sale_type[0], # Chung cư
    '41': sale_type[1], # Nhà riêng
    '325': sale_type[2], # biệt thự
    '163': sale_type[3], # Nhà mặt phố
    '575': sale_type[4], # Shophouse
    '40': sale_type[5], # Đất nền
    '283': sale_type[6], # Bán đất
    '44': sale_type[7], # Trang trại
    '45': sale_type[8], # kho
    '48': sale_type[9], # Khác
    '562': sale_type[10], # Khác
    '650': sale_type[10]
}


 Thiết kế hàm để lưu thông tin đặc trưng từ trang web

In [9]:
def ExtractDataFromLinkWithSelenium(url):
    driver = webdriver.Chrome()
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        #Create a dicitonary to store the feature
        features = {}

        # get the address
        address = soup.find(class_ = "re__pr-short-description js__pr-address")

        # Get the type
        regex = r'[\d]+'
        type = soup.find_all('script', type= 'text/javascript')
        for item in type:
            if (item.text):
                found = re.findall(regex, item.text)
                break;
        # If the address cannot be found, mean the data is invalide, ignore them.
        if address == None:
            return features
        else:
            features['Address'] = address.text
            features['Type'] = sale_type_dict[found[3]]

        # Get all the features about the attribute of each real estate
        feature_items = soup.find_all('div', class_ = 're__pr-specs-content-item')
        for item in feature_items:
            title = item.find(class_ = 're__pr-specs-content-item-title').text.strip()
            value = item.find(class_ = 're__pr-specs-content-item-value').text.strip()
            features[title] = value

        # Get the features about the posting date, expiry date and the ad code
        date_items = soup.find_all( 'div', class_ = "re__pr-short-info-item js__pr-config-item")
        for item in date_items:
            title = item.find('span', class_ = 'title').text.strip()
            value = item.find('span',class_ = 'value').text.strip()
            features[title] = value
    
        # Convert all the keys to the defined keys base on the dictionary.
        keys_to_change = list(features.keys())
        for key in keys_to_change:
            if key in key_map:
                features[key_map[key]] = features[key]
    finally:
        driver.quit()

    return features

Cào dữ liệu từ trang các url đã được lưu

In [ ]:
for url in urls[0:60000]:
        features = ExtractDataFromLinkWithSelenium(url)
        if features:
                new_row = pd.Series(features, index = fields)
                new_row_df = pd.DataFrame([new_row])
                df = pd.concat([df, new_row_df], ignore_index = True)
                #new_row_df.to_csv('../Data/real_estate_for_sale.csv', mode='a', header=False, index=False, encoding='utf-8')
        else:
                print ("Error")


In [11]:
df

,Address,Type,Area,Price,Bedroom,Toilet,Floor,Furniture,Direction,Legal,Posting date,Expiry date,Ad type,Ad code
0,"Dự án Central Premium, Phố Tạ Quang Bửu, Phườn...",Căn hộ chung cư,51 m²,"3,2 tỷ",1 phòng,NaN,NaN,Không nội thất.,NaN,NaN,06/06/2025,16/06/2025,Tin thường,40940633
1,"Đường Hương lộ 2, Phường Bình Trị Đông A, Bình...",Đất bán,56 m²,"4,89 tỷ",NaN,NaN,NaN,NaN,Tây,Sổ đỏ/ Sổ hồng,25/04/2025,05/05/2025,Tin thường,42827842
2,"Đường Độc Lập, Phường Tân Quý, Tân Phú, Hồ Chí...",Nhà mặt phố,270 m²,39 tỷ,NaN,NaN,2 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,02/06/2025,12/06/2025,Tin thường,42732012
3,"Đường Nguyễn Cư Trinh, Phường Nguyễn Cư Trinh,...",Nhà mặt phố,65 m²,40 tỷ,NaN,NaN,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,25/04/2025,05/05/2025,Tin thường,42827012
4,"Lucky Palace, Phan Văn Khỏe, 2, Quận 6, Hồ Chí...",Căn hộ chung cư,114 m²,6 tỷ,3 phòng,NaN,NaN,"- Nội thất cơ bản đầy đủ với điều hòa, tủ lạnh...",NaN,Sổ đỏ/ Sổ hồng,25/04/2025,05/05/2025,Tin thường,42635926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,"Đường Lê Văn Sỹ, Phường 1, Tân Bình, Hồ Chí Minh",Nhà riêng,45 m²,8 tỷ,4 phòng,NaN,4 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,09/06/2025,19/06/2025,Tin thường,42374969
1239,"Đường Sư Vạn Hạnh, Phường 12, Quận 10, Hồ Chí ...",Nhà riêng,109 m²,"23,8 tỷ",5 phòng,NaN,3 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,25/05/2025,04/06/2025,Tin thường,42342057
1240,"Đường Bạch Đằng, Phường 24, Bình Thạnh, Hồ Chí...",Nhà mặt phố,52 m²,"8,9 tỷ",NaN,NaN,3 tầng,Cơ bản,NaN,Sổ đỏ/ Sổ hồng,04/05/2025,14/05/2025,Tin thường,42571965
1241,"Đường Nguyễn Xí, Phường 13, Bình Thạnh, Hồ Ch...",Nhà riêng,"71,9 m²","9,2 tỷ",3 phòng,NaN,3 tầng,Cơ bản,NaN,Sổ đỏ/ Sổ hồng,04/05/2025,14/05/2025,Tin thường,42605110


### **4. Lưu thông tin vào file csv:**



In [12]:
df.to_csv('../Data/real_estate_for_sale.csv', mode='a', header=False, index=False, encoding='utf-8')

In [13]:
real_estate_df = pd.read_csv('../Data/real_estate_for_sale.csv')
real_estate_df

,Address,Type,Area,Price,Bedroom,Toilet,Floor,Furniture,Direction,Legal,Posting date,Expiry date,Ad type,Ad code
0,"Đường Lê Văn Việt, Phường Long Thạnh Mỹ, Quận ...",Nhà riêng,66 m²,Thỏa thuận,4 phòng,NaN,3 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,03/05/2025,13/05/2025,Tin thường,42376273
1,"Dự án Fuji Residence, Đường Đỗ Xuân Hợp, Phườn...",Căn hộ chung cư,55 m²,"2,7 tỷ",1 phòng,NaN,NaN,Đầy đủ,Tây - Bắc,Sổ đỏ/ Sổ hồng,03/05/2025,13/05/2025,Tin thường,41283793
2,"Đường Lê Văn Sỹ, Phường 1, Tân Bình, Hồ Chí Minh",Nhà riêng,45 m²,8 tỷ,4 phòng,NaN,4 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,08/05/2025,18/05/2025,Tin thường,42374969
3,"Đường Sư Vạn Hạnh, Phường 12, Quận 10, Hồ Chí ...",Nhà riêng,109 m²,"23,8 tỷ",5 phòng,NaN,3 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,15/05/2025,25/05/2025,Tin thường,42342057
4,"Đường Bạch Đằng, Phường 24, Bình Thạnh, Hồ Chí...",Nhà mặt phố,52 m²,"8,9 tỷ",NaN,NaN,3 tầng,Cơ bản,NaN,Sổ đỏ/ Sổ hồng,04/05/2025,14/05/2025,Tin thường,42571965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65316,"Đường Lê Văn Sỹ, Phường 1, Tân Bình, Hồ Chí Minh",Nhà riêng,45 m²,8 tỷ,4 phòng,NaN,4 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,09/06/2025,19/06/2025,Tin thường,42374969
65317,"Đường Sư Vạn Hạnh, Phường 12, Quận 10, Hồ Chí ...",Nhà riêng,109 m²,"23,8 tỷ",5 phòng,NaN,3 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,25/05/2025,04/06/2025,Tin thường,42342057
65318,"Đường Bạch Đằng, Phường 24, Bình Thạnh, Hồ Chí...",Nhà mặt phố,52 m²,"8,9 tỷ",NaN,NaN,3 tầng,Cơ bản,NaN,Sổ đỏ/ Sổ hồng,04/05/2025,14/05/2025,Tin thường,42571965
65319,"Đường Nguyễn Xí, Phường 13, Bình Thạnh, Hồ Ch...",Nhà riêng,"71,9 m²","9,2 tỷ",3 phòng,NaN,3 tầng,Cơ bản,NaN,Sổ đỏ/ Sổ hồng,04/05/2025,14/05/2025,Tin thường,42605110


In [ ]:
real_estate_df.shape()